#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [12]:
using Random, LinearAlgebra
p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
  1.25   0.0    0.25   0.25  0.0   0.0   0.0   0.0    0.0   -0.25
  0.0    1.25   0.0    0.0   0.0  -0.25  0.0  -0.25  -0.25   0.0
  0.25   0.0    1.25   0.25  0.0   0.0   0.0   0.0    0.0   -0.25
  0.25   0.0    0.25   1.25  0.0   0.0   0.0   0.0    0.0   -0.25
  0.0    0.0    0.0    0.0   1.0   0.0   0.0   0.0    0.0    0.0
  0.0   -0.25   0.0    0.0   0.0   1.25  0.0   0.25   0.25   0.0
  0.0    0.0    0.0    0.0   0.0   0.0   1.0   0.0    0.0    0.0
  0.0   -0.25   0.0    0.0   0.0   0.25  0.0   1.25   0.25   0.0
  0.0   -0.25   0.0    0.0   0.0   0.25  0.0   0.25   1.25   0.0
 -0.25   0.0   -0.25  -0.25  0.0   0.0   0.0   0.0    0.0    1.25

In [13]:
n = 1000 #Sample size

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Σ = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  1.2885      -0.00166515   0.240096   …  -0.0180624  -0.276094
 -0.00166515   1.19835      0.031256      -0.29201    -0.0353311
  0.240096     0.031256     1.28936       -0.0332284  -0.315479
  0.202755     0.0559613    0.302539       0.0364086  -0.265132
 -0.0204167    0.00384667   0.0133862     -0.0648273  -0.0207834
  0.0052062   -0.24765     -0.036967   …   0.217528    0.0277423
  0.0185172   -0.0404634   -0.0191132      0.0194558   0.0386565
  0.0123348   -0.203201     0.0341         0.252773    0.00421675
 -0.0180624   -0.29201     -0.0332284      1.31456    -0.0438309
 -0.276094    -0.0353311   -0.315479      -0.0438309   1.25593

In [14]:
show(stdout, "text/plain", Σ)

10×10 Matrix{Float64}:
  1.2885      -0.00166515   0.240096    0.202755    -0.0204167    0.0052062   0.0185172   0.0123348   -0.0180624  -0.276094
 -0.00166515   1.19835      0.031256    0.0559613    0.00384667  -0.24765    -0.0404634  -0.203201    -0.29201    -0.0353311
  0.240096     0.031256     1.28936     0.302539     0.0133862   -0.036967   -0.0191132   0.0341      -0.0332284  -0.315479
  0.202755     0.0559613    0.302539    1.26655      0.00563072   0.0309291  -0.0665495  -0.022145     0.0364086  -0.265132
 -0.0204167    0.00384667   0.0133862   0.00563072   1.04526      0.061717   -0.0174228  -0.0210354   -0.0648273  -0.0207834
  0.0052062   -0.24765     -0.036967    0.0309291    0.061717     1.3015      0.0775003   0.256675     0.217528    0.0277423
  0.0185172   -0.0404634   -0.0191132  -0.0665495   -0.0174228    0.0775003   1.03382     0.055277     0.0194558   0.0386565
  0.0123348   -0.203201     0.0341     -0.022145    -0.0210354    0.256675    0.055277    1.26787      0.

In [15]:
[x1 x2]

10×2 Matrix{Float64}:
  0.5   0.0
  0.0  -0.5
  0.5   0.0
  0.5   0.0
  0.0   0.0
  0.0   0.5
  0.0   0.0
  0.0   0.5
  0.0   0.5
 -0.5   0.0

In [16]:
[k, k]

2-element Vector{Int64}:
 4
 4

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [17]:
include("../julia/algorithm2.jl")

findmultPCs_deflation (generic function with 1 method)

In [18]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Σ, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 


          1 |                0.339 |                  2.00e+00 |      0.266 
          2 |                0.339 |                  2.00e+00 |      0.295 
          3 |                0.339 |                  2.00e+00 |      0.312 
          4 |                0.333 |                  1.00e-07 |      0.341 


          5 |                0.333 |                  1.00e-07 |      0.366 
          6 |                0.333 |                  1.00e-07 |      0.386 
          7 |                0.333 |                  1.00e-07 |      0.410 


10×2 Matrix{Float64}:
  0.0        0.461411
  0.469387   0.0
  0.0        0.532221
  0.0        0.48601
  0.0        0.0
 -0.508664   0.0
  0.0        0.0
 -0.487371   0.0
 -0.532359   0.0
  0.0       -0.517335

In [19]:
show(stdout, "text/plain", x_best)

10×2 Matrix{Float64}:
  0.0        0.461411
  0.469387   0.0
  0.0        0.532221
  0.0        0.48601
  0.0        0.0
 -0.508664   0.0
  0.0        0.0
 -0.487371   0.0
 -0.532359   0.0
  0.0       -0.517335

#### Step 3: C++ implementation

Applies the C++ code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

See https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/ for documentation on calling C code in Julia

In [20]:
ofv_best = Ref{Cdouble}(0)
violation_best = Ref{Cdouble}(0)
runtime = Ref{Cdouble}(0)
x_best = zeros(Cdouble, r * p)
ptr_best = pointer(x_best)
numIters = 20 
verbose = 1
violation_tolerance = 1e-4
ks_double = [convert(Float64, k), convert(Float64, k)]

@ccall "../c++/algorithm2.so".findmultPCs_deflation(
   ofv_best :: Ptr{Cdouble},
   violation_best :: Ptr{Cdouble},
   runtime :: Ptr{Cdouble},
   pointer(x_best) :: Ptr{Cdouble},
   p :: Cint,
   r :: Cint,
   pointer(reinterpret(Cdouble, reshape(Σ, p * p))) :: Ptr{Cdouble},
   pointer(reinterpret(Cdouble, reshape(ks_double, 1 * r))) :: Ptr{Cdouble},
   numIters :: Cint,
   verbose :: Cint,
   violation_tolerance :: Cdouble
)::Cvoid;

ofv_best = ofv_best[]::Float64
violation_best = violation_best[]::Float64
runtime = runtime[]::Float64
x_best = reshape(reinterpret(Float64, x_best), r, p)
println(x_best)

[0.0 0.46990273513678765 0.0 0.0 0.0 -0.5062973547312829 0.0 -0.4871532799105145 -0.5343557709761346 0.0; 0.4657641831995906 0.0 0.532672870435777 0.4828866931447743 0.0 0.0 0.0 0.0 0.0 -0.5158912485532513]
